데이터: https://www.code.go.kr/stdcode/regCodeL.do

In [126]:
import pandas as pd
import os
# from os import write
import locale
import re

In [127]:
# 경로명
path_name = '법정동코드_조회자료/'

In [128]:
# 파일 압축 해제
zip_name = '법정동코드_조회자료.zip'
os.system("unzip " + zip_name + " -d " + path_name)

256

In [129]:
sorted_filenames = sorted(os.listdir(path_name), key=lambda x: x.split('_')[2].split('.')[0])
print(sorted_filenames)

['법정동코드_조회자료_강원.xls', '법정동코드_조회자료_경기.xls', '법정동코드_조회자료_경남.xls', '법정동코드_조회자료_경북.xls', '법정동코드_조회자료_광주.xls', '법정동코드_조회자료_대구.xls', '법정동코드_조회자료_대전.xls', '법정동코드_조회자료_부산.xls', '법정동코드_조회자료_서울.xls', '법정동코드_조회자료_세종.xls', '법정동코드_조회자료_울산.xls', '법정동코드_조회자료_인천.xls', '법정동코드_조회자료_전남.xls', '법정동코드_조회자료_전북.xls', '법정동코드_조회자료_제주.xls', '법정동코드_조회자료_충남.xls', '법정동코드_조회자료_충북.xls']


In [130]:
city_id = 0
district_id = 0
town_id = 0

In [131]:
town_pattern = r'.*(읍|면|동|로)([1-9]가)?$'

In [132]:
f1 = open('DBEM_City_SQL.txt', 'w')
f2 = open('DBEM_District_SQL.txt', 'w')
f3 = open('DBEM_Town_SQL.txt', 'w')

In [133]:
# Daraframe형식으로 엑셀 파일 읽기
for file_name in sorted_filenames:
  df = pd.read_excel(path_name+file_name)

  # print(df.head())

  for code, region in zip(df['법정동코드'], df['법정동명']):
    if str(code)[2:] == "00000000":
      city_id += 1

      s = f'INSERT INTO city VALUES ({city_id}, \'{region}\', \'{code}\');\n'
      f1.write(s)

    elif region == "세종특별자치시":
      city_id += 1
      district_id += 1

      s = f'INSERT INTO city VALUES ({city_id}, \'{region}\', \'{code}\');\n'
      f1.write(s)

      s = f'INSERT INTO district VALUES ({district_id}, {city_id}, \'{region}\', \'{code}\');\n'
      f2.write(s)
      if (district_id % 5 == 0):
          f2.write('\n')

    elif region.split(' ')[0] == "세종특별자치시" and str(code)[4:] != "000000":
      if len(region.split(' ')) == 2:
        town_id += 1
        town = region.split(' ')[1]

        s = f'INSERT INTO town VALUES ({town_id}, {district_id}, \'{town}\', \'{code}\');\n'
        f3.write(s)
        if (town_id % 5 == 0):
            f3.write('\n')

    elif str(code)[5:] == "00000":
      district_id += 1
      district = region.split(' ', maxsplit=1)[1].rstrip()

      s = f'INSERT INTO district VALUES ({district_id}, {city_id}, \'{district}\', \'{code}\');\n'
      f2.write(s)
      if (district_id % 5 == 0):
          f2.write('\n')

    elif not region.split(' ')[-1].endswith('리'):
      town_id += 1
      towns = region.split(' ')
      town = next((t for t in towns if re.match(town_pattern, t)), None)

      s = f'INSERT INTO town VALUES ({town_id}, {district_id}, \'{town}\', \'{code}\');\n'
      f3.write(s)
      if (town_id % 5 == 0):
          f3.write('\n')

In [134]:
f1.close()
f2.close()
f3.close()

In [135]:
# 저장된 .txt 파일에서 한 줄 출력해 보기
data = open('DBEM_Town_SQL.txt', 'r', encoding="UTF8")
contents = data.readline()
print(contents)
data.close()

INSERT INTO town VALUES (1, 1, '봉의동', '5111010100');



In [136]:
print(city_id + district_id + town_id)

5341
